In [1]:
%matplotlib inline

In [2]:
import torch
import torch.optim as optim
import numpy as np

![rnn_architectures](../_static/rnn_architectures.png)

(ref: http://cs231n.stanford.edu/slides/2018/cs231n_2018_lecture10.pdf)

본 실습에서는 문자열 입력을 받아 문자열을 예측하므로, 5번째 그림(many to many)에 해당합니다.

## 입력 데이터

RNN에 문장을 입력하기 위해선 모든 문자를 텐서 형태로 바꿔주어야 합니다. 따라서 앞장의 경우와 마찬가지로 각 문자가 서로 다른 벡터로 매핑되도록 변환해줍니다. 아래는 그 예시입니다. 

In [3]:
sample = " Samsung Experts, AI Security"

문장 내 각 문자가 서로 다른 인덱스를 갖도록 dictionary 형태로 매핑

In [4]:
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'S': 0, 'i': 1, 'm': 2, ',': 3, 'g': 4, 'r': 5, 'e': 6, 'a': 7, 't': 8, 'n': 9, 'x': 10, 'A': 11, ' ': 12, 'E': 13, 'c': 14, 's': 15, 'p': 16, 'y': 17, 'u': 18, 'I': 19}


In [5]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [6]:
dic_size, hidden_size, learning_rate

(20, 20, 0.1)

dictionary 로 서로 다른 인덱스로 매핑된 각 문자 기반으로 입력 데이터를 벡터화 (one-hot encoding)

In [7]:
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

In [8]:
# transform as torch tensor variable
X = torch.tensor(x_one_hot).float()
Y = torch.tensor(y_data)

입력 데이터 확인

In [9]:
X.view(-1, dic_size), Y.view(-1)

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         

In [16]:
X.shape

torch.Size([1, 28, 20])

## RNN 선언

In [17]:
dic_size, hidden_size

(20, 20)

In [10]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

RNN의 아웃풋이 마지막 레이어의 가중치로부터 계산된 logit 값들이므로 손실 함수로 nn.CrossEntropyLoss 가 적합합니다.  
  
optimizer 로 optim.Adam 을 사용하도록 합니다.

In [11]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

입력 데이터에 대한 RNN 모델의 출력 확인

In [12]:
outputs, _status = rnn(X)

result = outputs.data.numpy().argmax(axis=2)
result_str = ''.join([char_set[c] for c in np.squeeze(result)])

print("prediction: ", result)
print("true Y: ", y_data)
print("prediction str: ", result_str)

prediction:  [[13  6  6 13  6 13  6  6 12  6 13 13  9  9  9 13  9 13 13 12 13 13 13  9
  13  9 13  9]]
true Y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  EeeEeEee eEEnnnEnEE EEEnEnEn


## 학습

각 학습 루프는 다음과 같습니다:

-  각 문자를 읽기

   -  다음 문자를 위한 은닉 상태 유지

-  목표와 최종 출력 비교
-  역전파
-  출력과 손실 반환

In [13]:
# start training
for i in range(70):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    
    print("Epoch :", i, "- loss: ", loss.item())
    print("pred_y: ", result)
    print("true_y: ", y_data)
    print("prediction str: ", result_str)
    print("========================")

Epoch : 0 - loss:  2.9766178131103516
pred_y:  [[13  6  6 13  6 13  6  6 12  6 13 13  9  9  9 13  9 13 13 12 13 13 13  9
  13  9 13  9]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  EeeEeEee eEEnnnEnEE EEEnEnEn
Epoch : 1 - loss:  2.650142192840576
pred_y:  [[ 7  7 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
  12 12  8 12]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  aa                        t 
Epoch : 2 - loss:  2.348005533218384
pred_y:  [[ 0  7 12 18 18  5  0 12  7  6 16  6  5  0  0  3  7 11 12 12  0  6 12 18
   5  1  0 17]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  Sa uurS aeperSS,aA  Se uriSy
Epoch : 3 - loss:  2.188678741455078
pred_y:  [[ 0  7  2 18 18  9  0 12 13  6  2  6  5  0 15  3  7 11 19 12  0  6  2  0
   

Epoch : 54 - loss:  1.3466976881027222
pred_y:  [[ 0  6  2 15 18  9  4 12 13 10 16  6  5  8 15  3 12 11 19 12  0  6 14 18
   5  1  8 17]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  Semsung Experts, AI Security
Epoch : 55 - loss:  1.3457335233688354
pred_y:  [[ 0  6  2 15 18  9  4 12 13 10 16  6  5  8 15  3 12 11 19 12  0  6 14 18
   5  1  8 17]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  Semsung Experts, AI Security
Epoch : 56 - loss:  1.3447352647781372
pred_y:  [[ 0  6  2 15 18  9  4 12 13 10 16  6  5  8 15  3 12 11 19 12  0  6 14 18
   5  1  8 17]]
true_y:  [[0, 7, 2, 15, 18, 9, 4, 12, 13, 10, 16, 6, 5, 8, 15, 3, 12, 11, 19, 12, 0, 6, 14, 18, 5, 1, 8, 17]]
prediction str:  Semsung Experts, AI Security
Epoch : 57 - loss:  1.3439263105392456
pred_y:  [[ 0  6  2 15 18  9  4 12 13 10 16  6  5  8 15  3 12 11 19 12  0  6 14

------------------------------------------------------------------------------------------




# Practice

#### Q1. 입력 문장을 " I think therefore I am." 로 변경한 뒤, 본 실습을 수행해볼 것.

#### Q2. 학습이 잘 되는 지 확인하시오.

#### Q2. 학습이 잘 되지 않는다면 RNN 모듈을 LSTM 모듈로 바꿔 학습을 수행해보시오.

------------------------------------------------------------------------------------------




__(ref: https://github.com/deeplearningzerotoall/PyTorch)__